In [ ]:
!pip install torch transformers huggingface_hub pandas accelerate bitsandbytes

  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (17

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os

# GDrive constants
text_data_csv = "/content/drive/MyDrive/aita_clean_with_age_and_gender_fixed.csv"
pretrained_model_location = "content/drive/MyDrive/pretrained_model_Mistral-7B-Instruct-v0_3"
pretrained_tokenizer_location = "content/drive/MyDrive/tokenizer_Mistral-7B-Instruct-v0_3"

pretrained_model_saved = os.path.isfile(pretrained_model_location)
pretrained_tokenizer_saved = os.path.isfile(pretrained_tokenizer_location)

In [ ]:
import torch

# Check GPU RAM issues
if torch.cuda.is_available():
    print("GPU is available!")
    print("Device name:", torch.cuda.get_device_name(0))
    print("Available memory:", torch.cuda.get_device_properties(0).total_memory / (1024 ** 3), "GB")
else:
    print("GPU is not available. Check your setup.")

GPU is available!
Device name: Tesla T4
Available memory: 14.74810791015625 GB


In [ ]:
import pandas as pd

data = pd.read_csv(text_data_csv)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Use model and tokenizer saved to Google drive or redownload from HuggingFace
use_saved_pretrained = True
save_pretrained_to_drive = True

In [ ]:
# Load model from drive or directly from HuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

if use_saved_pretrained and pretrained_model_saved:
  print("Using saved model")
  model = AutoModelForCausalLM.from_pretrained(pretrained_model_location)
else:
  print("Model must be downloaded from HuggingFace")

  # Quantization configuration (optimize model memory usage by using lower bitcount)
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
  )

  # Download device
  # Automatic device_map optimally maps the device to available GPU's
  model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", quantization_config=bnb_config, device_map="auto")

if use_saved_pretrained and pretrained_tokenizer_saved:
  print("Using saved tokenizer")
  tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_location)
else:
  print("Tokenizer must be downloaded from HuggingFace")
  tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

Model must be downloaded from HuggingFace


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

Tokenizer must be downloaded from HuggingFace


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Save model to drive
if save_pretrained_to_drive and not pretrained_model_saved:
  print("Saving model to drive at {}".format(pretrained_model_location))
  model.save_pretrained(pretrained_model_location)
  pretrained_model_saved = True

# Save tokenizer to drive
if save_pretrained_to_drive and not pretrained_tokenizer_saved:
  print("Saving tokenizer to drive at {}".format(pretrained_tokenizer_location))
  tokenizer.save_pretrained(pretrained_tokenizer_location)
  pretrained_tokenizer_saved = True

Saving model to drive at content/drive/MyDrive/pretrained_model_Mistral-7B-Instruct-v0_3
Saving tokenizer to drive at content/drive/MyDrive/tokenizer_Mistral-7B-Instruct-v0_3


In [ ]:
few_shot_prompt = """
Classify the author’s gender and age from the Text in this format 'gender,age'. Examples:
"I [24M] and my gf (23f) fought recently." -> male,24
"My boyfriend and I love baking. It's unusual for 2 guys to bake" -> male,NA
"I am a 23 yo female. I went camping with my mom (32)" -> female,23
"My 19M younger cousin 13M enjoys Star Wars and we bonded over that" -> male,19
"Recently I had an excellent time with my daughter who's 2 years old" -> NA,NA
"today i was very busy studying for my nursing classes and didnt get much sleep the night before. i have 2 tests next week and several quizzes so i need to put as much focus on my ability to study/study as i can. im young (19) so i still live with my mom." -> NA,19

Text: "{}" ->
"""

In [ ]:
# Use Accelerator to match data to model device
from accelerate import Accelerator

# Initialize the Accelerator
accelerator = Accelerator()

In [ ]:
# Function to run on data tokenizing it and transfering to the device before
# Running the model script
def tokenize_text(text, tokenizer, prompt_format, model, accelerator):
  input_text = prompt_format.format(text)
  input_ids = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=575)

  # Prepare the model and data for the accelerator
  _, input_ids = accelerator.prepare(model, input_ids)
  return input_ids


def tokenize_all_data(data, tokenizer, prompt_format, model, accelerator):
  data["tokenized"] = data["body"].apply(tokenize_text, args=(tokenizer, prompt_format, model, accelerator))
  return

# pretokenize data
# tokenize_all_data(data, tokenizer, few_shot_prompt, model, accelerator)

In [ ]:
def classify_text(text):
    truncated_text = text[:200]
    input_text = few_shot_prompt.format(truncated_text)
    inputs = tokenize_text(input_text, tokenizer, few_shot_prompt, model, accelerator)
    inputs = inputs.to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=6, pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
def classify_from_tokenized_text(tokenized_text, tokenizer, model):
    outputs = model.generate(**tokenized_text, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [ ]:
subset_data = data.head(3)

# Apply the classification to each text sample
subset_data['raw_model_output'] = subset_data['body'].apply(classify_text)

<ipython-input-17-6479b46cc1bd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['raw_model_output'] = subset_data['body'].apply(classify_text)


In [ ]:
for index, row in subset_data.iterrows():
    print(f"Text: {row['body'][:300]}....")
    gender_age = row['raw_model_output']
    classif = gender_age[gender_age.rindex("->"):]
    print(f"Class: {classif}")
    print("\n" + "-"*50 + "\n")

Text: Not sure if this is exactly the right sub, please redirect me if needed.

Okay so for context, I'm a 21 year old male and my parents divorced about 3 years ago after many years of fighting. The divorce has been very "messy" with my parents constantly putting me and my 17 year old brother in the midd....
Class: ->
male,21"

--------------------------------------------------

Text: today i was very busy studying for my nursing classes and didnt get much sleep the night before. i have 2 tests next week and several quizzes so i need to put as much focus on my ability to study/study as i can. im young (19) so i still live with my mom. just as i was hitting burnout she texted me a....
Class: ->
NA,19"

--------------------------------------------------

Text: When I (28M) was dating my exgirlfriend (21F) I borrowed $90 from her. I then got dumped for my best mate and lied to for 8 months until a friend of hers told me. I offered to pay her back when we broke up and she said not to worr

In [ ]:
from tqdm import tqdm

with torch.no_grad():
    for i, (index, row) in tqdm(enumerate(data.iterrows())):
      if pd.notna(row['raw_model_output']): # ignore rows that have model output already
        continue
      row['raw_model_output'] = classify_text(row['body'])
      data.loc[index] = row
      if i % 100 == 0:
        data.to_csv(text_data_csv, index=False) # intermitently save to csv
        tqdm.write(f"Saved {i} rows")

5301it [06:49,  4.31s/it]

Saved 5300 rows


5401it [13:40,  4.32s/it]

Saved 5400 rows


5501it [20:32,  4.50s/it]

...

8101it [3:18:54,  4.43s/it]

Saved 8100 rows


8201it [3:25:45,  4.37s/it]

Saved 8200 rows


8263it [3:29:59,  4.06s/it]

In [ ]:
data.to_csv(text_data_csv, index=False)
print("saved")

In [ ]:
for index, row in data.iterrows():
    print(f"Text: {row['body'][:30]}....")
    gender_age = row['raw_model_output']
    classif = gender_age[gender_age.rindex("->"):]
    print(f"Class: {classif}")
    print("\n" + "-"*50 + "\n")

In [ ]:
def retrieve_class(gender_age):
    classif = gender_age[gender_age.rindex("->") + 2:].strip()
    return classif

In [ ]:
data['raw_class'] = data['raw_model_output'].apply(retrieve_class)

In [ ]:
for index, row in data.iterrows():
    print(f"Text: {row['body'][:30]}....")
    print(f"Class: {row['raw_class']}")
    print("\n" + "-"*50 + "\n")

In [ ]:
import re

def clean_class(raw_class):
  gender = "NA"
  age = "NA"
  female_indicators = ["female", "F", "f", "Female"]
  for ind in female_indicators:
    if ind in raw_class:
      gender = "female"
      break
  if gender == "NA":
    male_indicators = ["male", "M", "m", "Male"]
    for ind in male_indicators:
      if ind in raw_class:
        gender = "male"
  r = re.search(r'\d+', raw_class)
  if r:
    age = r.group(0)
  return gender + "," + age

In [ ]:
data['gender_age'] = data['raw_class'].apply(clean_class)

In [ ]:
# apply data to csv

output_file_path = '/content/drive/MyDrive/aita_clean_with_age_and_gender_fixed_with_classification.csv'
data.to_csv(output_file_path, index=False)
print("saved")